<a href="https://colab.research.google.com/github/zebaayaz/mood_tracker/blob/master/Assignment1(latest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

from sklearn.model_selection import train_test_split

import numpy as np


In [0]:
dataset =pd.read_csv('housing.csv')

dataset= dataset.dropna ()

print("Here are the first ten records of housing dataset:")
dataset.head(10)


In [0]:
import matplotlib.pyplot as plt
dataset.plot(subplots='true')




In [0]:
# We will predict the "median_house_value" column
 Y = dataset['median_house_value']

 X = dataset.loc[:,'longitude':'median_income']


In [0]:
# Splits the dataset so 70% is used for training and 30% for testing
 x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

# Converts the datasets to numpy arrays to work with our PyTorch model
x_train_np = x_train.to_numpy()
y_train_np = y_train.to_numpy()

# Convert the testing data 
x_test_np = x_test.to_numpy()
y_test_np = y_test.to_numpy()


In [0]:
# Import the pytorch library 
import torch

# Import the 1D convolution layer
# Since we’re inputting a 1-dimensional row of data, we can’t use 2D or 3D
from torch.nn import Conv1d

# Import the max pooling layer
from torch.nn import MaxPool1d

# Import the flatten layer
from torch.nn import Flatten

# Import the linear layer
from torch.nn import Linear

# Import the ReLU activation function 
from torch.nn.functional import relu

# Import the DataLoader and TensorDataset libraries from PyTorch # to work with our datasets
from torch.utils.data import DataLoader, TensorDataset


In [0]:
# Our class MUST be a subclass of torch.nn.Module
class CnnRegressor(torch.nn.Module):

# Define the initialization method
  def __init__(self, batch_size, inputs, outputs): 
    super(CnnRegressor, self).__init__()  
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs


    self.input_layer = Conv1d(inputs, batch_size, 1)
    self.max_pooling_layer = MaxPool1d(1)  
    self.conv_layer = Conv1d(batch_size, 128, 1)
    self.flatten_layer = Flatten()
    self.linear_layer = Linear(128, 64)
    self.flatten_layer = Flatten()
    self.output_layer = Linear(64, outputs)

  def feed(self,input):
    input = input.reshape((self.batch_size, self.inputs, 1))
    output = relu(self.input_layer(input))
    output = self.max_pooling_layer(output)
    output = relu(self.conv_layer(output))
    output = self.flatten_layer(output)
    output = self.linear_layer(output)
    
    output = self.output_layer(output)
    return output


In [0]:
# Our class MUST be a subclass of torch.nn.Module
class CnnRegressor(torch.nn.Module):

# Define the initialization method
  def __init__(self, batch_size, inputs, outputs): 
    super(CnnRegressor, self).__init__()  
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs


    self.input_layer = Conv1d(inputs, batch_size, 1)
    self.max_pooling_layer = MaxPool1d(1)  
    self.conv_layer = Conv1d(batch_size, 128, 1)
    self.flatten_layer = Flatten()
    self.linear_layer = Linear(128, 64)
    self.flatten_layer = Flatten()
    self.output_layer = Linear(64, outputs)

  def feed(self,input):
    input = input.reshape((self.batch_size, self.inputs, 1))
    output = relu(self.input_layer(input))
    output = self.max_pooling_layer(output)
    output = relu(self.conv_layer(output))
    output = self.flatten_layer(output)
    output = self.linear_layer(output)
    output = self.flatten_layer(output)
    output = self.output_layer(output)
    return output


In [0]:

from torch.optim import SGD
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score
   


In [0]:

batch_size = 60

model = CnnRegressor(batch_size, X.shape[1], 1)

model.cuda()


In [0]:

def model_loss(model, dataset, train = False, optimizer = None):
  performance = L1Loss()
  score_metric = R2Score()
  avg_loss = 0
  avg_score = 0
  count = 0
  for input, output in iter(dataset):
    predictions = model.feed(input)
    loss = performance(predictions, output)
    score_metric.update([predictions, output]) 
    score = score_metric.compute()
    if(train):
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    avg_loss += loss.item()
    avg_score += score
    count += 1
  return avg_loss / count, avg_score / count



In [0]:
epochs = 150
optimizer = SGD(model.parameters(), lr=1e-5)

inputs = torch.from_numpy(x_train_np).cuda().float()
outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0], 1)).cuda().float()

tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

for epoch in range(epochs):
  avg_loss, avg_r2_score = model_loss(model, loader, train=True, optimizer=optimizer)
  print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\tR^2 Score = " + str(avg_r2_score))



In [0]:

inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0], 1)).cuda().float()

# Create a DataLoader instance to work with our batches
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

# Output the average performance of the model 
avg_loss, avg_r2_score = model_loss(model, loader)
print("The model's L1 loss is: " + str(avg_loss))
print("The model's R^2 score is: " + str(avg_r2_score))
